In [1]:
import numpy as np
from pycalphad import Model, Database, calculate, equilibrium
import pycalphad.variables as v

#dbf = Database('2016-08-10-AlGdMgand18RLPSO-for 3d plot.tdb')
dbf = Database('alfe_sei.TDB')
models = {key: Model(dbf, ['AL', 'FE', 'VA'], key) for key in dbf.phases.keys()}

In [14]:
#Set compiler directives (cf. http://docs.cython.org/src/reference/compilation.html)
%load_ext cython
from Cython.Compiler.Options import _directive_defaults

_directive_defaults['linetrace'] = True
_directive_defaults['binding'] = True

The cython extension is already loaded. To reload it, use:
  %reload_ext cython


In [15]:
%%cython -a -f --compile-args=-DCYTHON_TRACE=1
import pycalphad.variables as v
from pycalphad.core.utils import unpack_kwarg
from pycalphad.core.utils import unpack_condition, unpack_phases
from pycalphad import calculate, Model
from pycalphad.constraints import mole_fraction
from pycalphad.core.lower_convex_hull import lower_convex_hull
from pycalphad.core.sympydiff_utils import build_functions as compiled_build_functions
from pycalphad.core.constants import MIN_SITE_FRACTION
from pycalphad.core.eqsolver import _solve_eq_at_conditions
from sympy import Add, Symbol
import dask
from dask import delayed
import dask.multiprocessing, dask.async
from xarray import Dataset
import numpy as np
cimport numpy as np
from collections import namedtuple, OrderedDict
from datetime import datetime
from pycalphad.core.eqsolver import *
from pycalphad.core.eqsolver import _build_multiphase_gradient, \
    _build_multiphase_system, _compute_constraints, _compute_phase_dof, remove_degenerate_phases
cimport cython

def _solve_eq_at_conditions(dbf, comps, properties, phase_records, callable_dict, conds_keys, verbose):
    """
    Compute equilibrium for the given conditions.
    This private function is meant to be called from a worker subprocess.
    For that case, usually only a small slice of the master 'properties' is provided.
    Since that slice will be copied, we also return the modified 'properties'.

    Parameters
    ----------
    dbf : Database
        Thermodynamic database containing the relevant parameters.
    comps : list
        Names of components to consider in the calculation.
    properties : Dataset
        Will be modified! Thermodynamic properties and conditions.
    phase_records : dict of PhaseRecord
        Details on phase callables.
    callable_dict : dict of callable
        Objective functions for each phase.
    conds_keys : list of str
        List of conditions axes in dimension order.
    verbose : bool
        Print details.

    Returns
    -------
    properties : Dataset
        Modified with equilibrium values.
    """
    cdef:
        double indep_sum
        int num_phases, num_vars, cur_iter, old_phase_length, new_phase_length, var_idx, sfidx, pfidx, m, n
        np.ndarray[ndim=1, dtype=np.float64_t] gradient_term, p_y, l_constraints, step
        np.ndarray[ndim=1, dtype=np.float64_t] site_fracs, candidate_site_fracs, l_multipliers, new_l_multipliers, candidate_phase_fracs, phase_fracs
        np.ndarray[ndim=2, dtype=np.float64_t] l_hessian, ymat, zmat, qmat, rmat, constraint_jac
    # Factored out via profiling
    prop_MU_values = properties['MU'].values
    prop_NP_values = properties['NP'].values
    prop_Phase_values = properties['Phase'].values
    prop_X_values = properties['X'].values
    prop_Y_values = properties['Y'].values
    prop_GM_values = properties['GM'].values

    it = np.nditer(prop_GM_values, flags=['multi_index'])

    #if verbose:
    #    print('INITIAL CONFIGURATION')
    #    print(properties.MU)
    #    print(properties.Phase)
    #    print(properties.NP)
    #    print(properties.X)
    #    print(properties.Y)
    #    print('---------------------')
    while not it.finished:
        # A lot of this code relies on cur_conds being ordered!
        cur_conds = OrderedDict(zip(conds_keys,
                                    [np.asarray(properties['GM'].coords[b][a], dtype=np.float)
                                     for a, b in zip(it.multi_index, conds_keys)]))
        if len(cur_conds) == 0:
            cur_conds = properties['GM'].coords
        # sum of independently specified components
        indep_sum = np.sum([float(val) for i, val in cur_conds.items() if i.startswith('X_')])
        if indep_sum > 1:
            # Sum of independent component mole fractions greater than one
            # Skip this condition set
            # We silently allow this to make 2-D composition mapping easier
            prop_MU_values[it.multi_index] = np.nan
            prop_NP_values[it.multi_index + np.index_exp[:len(phases)]] = np.nan
            prop_Phase_values[it.multi_index + np.index_exp[:len(phases)]] = ''
            prop_X_values[it.multi_index + np.index_exp[:len(phases)]] = np.nan
            prop_Y_values[it.multi_index] = np.nan
            prop_GM_values[it.multi_index] = np.nan
            it.iternext()
            continue
        dependent_comp = set(comps) - set([i[2:] for i in cur_conds.keys() if i.startswith('X_')]) - {'VA'}
        if len(dependent_comp) == 1:
            dependent_comp = list(dependent_comp)[0]
        else:
            raise ValueError('Number of dependent components different from one')
        # chem_pots = OrderedDict(zip(properties.coords['component'].values, properties['MU'].values[it.multi_index]))
        # Used to cache generated mole fraction functions
        mole_fractions = {}
        for cur_iter in range(MAX_SOLVE_ITERATIONS):
            # print('CUR_ITER:', cur_iter)
            phases = list(prop_Phase_values[it.multi_index])
            if '' in phases:
                old_phase_length = phases.index('')
            else:
                old_phase_length = -1
            remove_degenerate_phases(prop_Phase_values[it.multi_index], prop_X_values[it.multi_index],
                                     prop_Y_values[it.multi_index], prop_NP_values[it.multi_index])
            phases = list(prop_Phase_values[it.multi_index])
            if '' in phases:
                new_phase_length = phases.index('')
            else:
                new_phase_length = -1
            # Are there removed phases?
            if '' in phases:
                num_phases = phases.index('')
            else:
                num_phases = len(phases)
            if num_phases == 0:
                raise ValueError('Zero phases are left in the system')
            zero_dof = np.all(
                (prop_Y_values[it.multi_index] == 1.) | np.isnan(prop_Y_values[it.multi_index]))
            if (num_phases == 1) and zero_dof:
                # Single phase with zero internal degrees of freedom, can't do any refinement
                # TODO: In the future we may be able to refine other degrees of freedom like temperature
                # Chemical potentials have no meaning for this case
                prop_MU_values[it.multi_index] = np.nan
                break
            phases = prop_Phase_values[it.multi_index + np.index_exp[:num_phases]]
            # num_sitefrac_bals = sum([len(dbf.phases[i].sublattices) for i in phases])
            # num_mass_bals = len([i for i in cur_conds.keys() if i.startswith('X_')]) + 1
            phase_fracs = prop_NP_values[it.multi_index + np.index_exp[:len(phases)]]
            phase_dof = [len(set(phase_records[name].variables) - {v.T, v.P}) for name in phases]
            # Flatten site fractions array and remove nan padding
            site_fracs = prop_Y_values[it.multi_index].ravel()
            # That *should* give us the internal dof
            # This may break if non-padding nan's slipped in from elsewhere...
            site_fracs = site_fracs[~np.isnan(site_fracs)]
            site_fracs[site_fracs < MIN_SITE_FRACTION] = MIN_SITE_FRACTION
            if len(site_fracs) == 0:
                print(properties)
                raise ValueError('Site fractions are invalid')
            phase_fracs[phase_fracs < MIN_SITE_FRACTION] = MIN_SITE_FRACTION
            var_idx = 0
            for name in phases:
                for idx in range(len(dbf.phases[name].sublattices)):
                    active_in_subl = set(dbf.phases[name].constituents[idx]).intersection(comps)
                    for ais in range(len(active_in_subl)):
                        site_fracs[var_idx + ais] = site_fracs[var_idx + ais] / sum(site_fracs[var_idx:var_idx + len(active_in_subl)])
                    var_idx += len(active_in_subl)
            l_constraints, constraint_jac, constraint_hess  = \
                _compute_constraints(dbf, comps, phases, cur_conds, site_fracs, phase_fracs, phase_records, mole_fractions=mole_fractions)
            # Reset Lagrange multipliers if active set of phases change
            if cur_iter == 0 or (old_phase_length != new_phase_length) or np.any(np.isnan(l_multipliers)):
                l_multipliers = np.zeros(l_constraints.shape[0])
            qmat, rmat = np.linalg.qr(constraint_jac.T, mode='complete')
            m = rmat.shape[1]
            n = qmat.shape[0]
            # Construct orthonormal basis for the constraints
            ymat = qmat[:, :m]
            zmat = qmat[:, m:]
            # Equation 18.14a in Nocedal and Wright
            p_y = np.linalg.solve(-np.dot(constraint_jac, ymat), l_constraints)
            num_vars = len(site_fracs) + len(phases)
            l_hessian, gradient_term = _build_multiphase_system(dbf, comps, phases, cur_conds, site_fracs, phase_fracs,
                                                                l_constraints, constraint_jac, constraint_hess,
                                                                l_multipliers, callable_dict, phase_records)
            if np.any(np.isnan(l_hessian)):
                print('Invalid l_hessian')
                l_hessian[:,:] = np.eye(l_hessian.shape[0])
            if np.any(np.isnan(gradient_term)):
                raise ValueError('Invalid gradient_term')
            # Equation 18.18 in Nocedal and Wright
            if m != n:
                if np.any(np.isnan(zmat)):
                    raise ValueError('Invalid zmat')
                try:
                    p_z = np.linalg.solve(np.dot(np.dot(zmat.T, l_hessian), zmat),
                                          -np.dot(np.dot(np.dot(zmat.T, l_hessian), ymat), p_y) - np.dot(zmat.T, gradient_term))
                except np.linalg.LinAlgError:
                    p_z = np.zeros(zmat.shape[1], dtype=np.float)
                step = np.dot(ymat, p_y) + np.dot(zmat, p_z)
            else:
                step = np.dot(ymat, p_y)
            old_energy = copy.deepcopy(prop_GM_values[it.multi_index])
            old_chem_pots = copy.deepcopy(prop_MU_values[it.multi_index])
            candidate_site_fracs = np.empty_like(site_fracs)
            candidate_phase_fracs = np.empty_like(phase_fracs)
            for sfidx in range(candidate_site_fracs.shape[0]):
                candidate_site_fracs[sfidx] = min(max(site_fracs[sfidx] + step[sfidx], MIN_SITE_FRACTION), 1)

            for pfidx in range(candidate_phase_fracs.shape[0]):
                candidate_phase_fracs[pfidx] = min(max(phase_fracs[pfidx] + step[candidate_site_fracs.shape[0] + pfidx], 0), 1)
            candidate_l_constraints, candidate_constraint_jac, candidate_constraint_hess = \
                _compute_constraints(dbf, comps, phases, cur_conds,
                                     candidate_site_fracs, candidate_phase_fracs, phase_records, mole_fractions=mole_fractions)
            candidate_energy, candidate_gradient_term = \
                _build_multiphase_gradient(dbf, comps, phases, cur_conds, candidate_site_fracs,
                                           candidate_phase_fracs, candidate_l_constraints,
                                           candidate_constraint_jac, l_multipliers,
                                           callable_dict, phase_records)
            # We updated degrees of freedom this iteration
            new_l_multipliers = np.linalg.solve(np.dot(constraint_jac, ymat).T,
                                                np.dot(ymat.T, gradient_term + np.dot(l_hessian, step)))
            np.clip(new_l_multipliers, -MAX_ABS_LAGRANGE_MULTIPLIER, MAX_ABS_LAGRANGE_MULTIPLIER,
                    out=new_l_multipliers)
            # XXX: Should fix underlying numerical problem at edges of composition space instead of working around
            if np.any(np.isnan(new_l_multipliers)):
                print('WARNING: Unstable Lagrange multipliers: ', new_l_multipliers)
                # Equation 18.16 in Nocedal and Wright
                # This method is less accurate but more stable
                new_l_multipliers = np.dot(np.dot(np.linalg.inv(np.dot(candidate_constraint_jac,
                                                                       candidate_constraint_jac.T)),
                                           candidate_constraint_jac), candidate_gradient_term)
                np.clip(new_l_multipliers, -MAX_ABS_LAGRANGE_MULTIPLIER, MAX_ABS_LAGRANGE_MULTIPLIER,
                    out=new_l_multipliers)
            l_multipliers = new_l_multipliers
            if np.any(np.isnan(l_multipliers)):
                print('Invalid l_multipliers after recalculation', l_multipliers)
                l_multipliers[:] = 0
            if verbose:
                print('NEW_L_MULTIPLIERS', l_multipliers)
            num_mass_bals = len([i for i in cur_conds.keys() if i.startswith('X_')]) + 1
            chemical_potentials = l_multipliers[sum([len(dbf.phases[i].sublattices) for i in phases]):
                                                sum([len(dbf.phases[i].sublattices) for i in phases]) + num_mass_bals]
            prop_MU_values[it.multi_index] = chemical_potentials
            prop_NP_values[it.multi_index + np.index_exp[:len(phases)]] = candidate_phase_fracs
            prop_X_values[it.multi_index + np.index_exp[:len(phases)]] = 0
            prop_GM_values[it.multi_index] = candidate_energy
            var_offset = 0
            for phase_idx in range(len(phases)):
                prop_Y_values[it.multi_index + np.index_exp[phase_idx, :phase_dof[phase_idx]]] = \
                    candidate_site_fracs[var_offset:var_offset + phase_dof[phase_idx]]
                for comp_idx, comp in enumerate([c for c in comps if c != 'VA']):
                    prop_X_values[it.multi_index + np.index_exp[phase_idx, comp_idx]] = \
                        mole_fractions[(phases[phase_idx], comp)][0](
                            [candidate_site_fracs[var_offset:var_offset + phase_dof[phase_idx]]])
                var_offset += phase_dof[phase_idx]

            properties.attrs['solve_iterations'] += 1
            total_comp = np.nansum(prop_NP_values[it.multi_index][..., np.newaxis] * \
                                   prop_X_values[it.multi_index], axis=-2)
            driving_force = (prop_MU_values[it.multi_index] * total_comp).sum(axis=-1) - \
                             prop_GM_values[it.multi_index]
            driving_force = np.squeeze(driving_force)
            if verbose:
                print('Chem pot progress', prop_MU_values[it.multi_index] - old_chem_pots)
                print('Energy progress', prop_GM_values[it.multi_index] - old_energy)
                print('Driving force', driving_force)
            no_progress = np.abs(prop_MU_values[it.multi_index] - old_chem_pots).max() < 0.01
            no_progress &= np.abs(prop_GM_values[it.multi_index] - old_energy) < MIN_SOLVE_ENERGY_PROGRESS
            if no_progress and np.abs(driving_force) > MAX_SOLVE_DRIVING_FORCE:
                print('Driving force failed to converge: {}'.format(cur_conds))
                prop_MU_values[it.multi_index] = np.nan
                prop_NP_values[it.multi_index] = np.nan
                prop_X_values[it.multi_index] = np.nan
                prop_Y_values[it.multi_index] = np.nan
                prop_GM_values[it.multi_index] = np.nan
                prop_Phase_values[it.multi_index] = ''
                break
            elif no_progress:
                if verbose:
                    print('No progress')
                num_mass_bals = len([i for i in cur_conds.keys() if i.startswith('X_')]) + 1
                chemical_potentials = l_multipliers[sum([len(dbf.phases[i].sublattices) for i in phases]):
                                                    sum([len(dbf.phases[i].sublattices) for i in phases]) + num_mass_bals]
                prop_MU_values[it.multi_index] = chemical_potentials
                break
            elif (not no_progress) and cur_iter == MAX_SOLVE_ITERATIONS-1:
                print('Failed to converge: {}'.format(cur_conds))
                prop_MU_values[it.multi_index] = np.nan
                prop_NP_values[it.multi_index] = np.nan
                prop_X_values[it.multi_index] = np.nan
                prop_Y_values[it.multi_index] = np.nan
                prop_GM_values[it.multi_index] = np.nan
                prop_Phase_values[it.multi_index] = ''
        it.iternext()
    return properties

In [19]:
%lprun -f _solve_eq_at_conditions equilibrium(dbf, ['AL', 'FE', 'VA'], list(dbf.phases.keys()), {v.T: 700, v.X('AL'): (0,1,0.02), v.P: 101325}, model=models, solve_eq_at_conditions=_solve_eq_at_conditions)